# BeautifulSoup - Tags

In [ ]:
import requests
from bs4 import BeautifulSoup
from requests.exceptions import ConnectionError

In [ ]:
def download_html(url, numero_tentativas=2):
    print("Realizando o download da página:", url)
    try:
        req = requests.get(url)
        if req.status_code != 200:
            if numero_tentativas > 0:
                print("Não foi possível realizar o download. Erro:", req.status_code)
                print("\nRealizando nova tentativa:")
                return download_html(url, numero_tentativas - 1)
            else:
                print("Número de tentativas excedidas. Erro: {}".format(req.status_code))
                html = None
                return html
        html = req.text
        return html
    except ConnectionError as e:
        print("Erro no download:", e)
        html = None

In [ ]:
html = download_html("http://pythonscraping.com/pages/page3.html")

In [ ]:
bs = BeautifulSoup(html, "html.parser")

In [ ]:
bs.find({"span"})

In [ ]:
bs.findAll(name={"span"})

Além do parâmetro ```name``` utilizado anteriormente, é possível utilizar os seguintes parâmetros:
- ```attrs={}``` – Caso o nome a ser procurado seja uma palavra reservada do Python, utiliza-se o atributo attrs.
- ```recursive``` – Se a recursão for definida como True, a função descerá aos filhos e aos filhos dos filhos procurando tags que coincidam com seus parâmetros.
- ```text``` – procurar ocorrências de acordo com o conteúdo de texto das tags.
- ```limit``` – é utilizado no findAll e recupera os n primeiros itens da página.

In [ ]:
for item in bs.findAll({"span"}):
    print("-->", item.text)

## Lidando com filhos e outros descendentes

Se você o escrevesse usando a função ```descendants()``` em vez da função ```children()```, outras tags seriam encontradas (img, span, entre outros). **É muito importante diferenciar filhos e descendentes!**

Para listar as linhas de produtos da tabela ```giftList```, temos que criar um iterador e imprimir todos os filhos de uma tag.

In [ ]:
bs.find("table", {"id":"giftList"}).children

In [ ]:
# Recupera todas as linhas, inclusive a linha de titulo
for filho in bs.find("table", {"id":"giftList"}).children:
    print(filho)

## Lidando com irmãos
Para exibir todas as linhas de produtos da tabela.

In [ ]:
for irmao in bs.find("table", {"id":"giftList"}).tr.next_siblings:
    print(irmao)

### Acessando os elementos e estruturando com o Pandas


In [ ]:
import pandas as pd

In [ ]:
aux = []

for filho in bs.find("table", {"id":"giftList"}).children:
    aux.append(filho)

In [ ]:
aux_final = []
for i in range(1, len(aux), 2):
    aux_final.append(aux[i])

In [ ]:
colunas = [th.text.replace('\n', '') for th in aux_final[0].findAll('th')]
print(colunas)

In [ ]:
estrutura = {}
# Remove a coluna de Imagem
for col in colunas[:-1]:
    estrutura[col] = []

In [ ]:
for item in aux_final[1:]:
    aux = [td.text.replace('\n', '') for td in item.findAll('td')]
    estrutura['Item Title'].append(aux[0])
    estrutura['Description'].append(aux[1])
    str_aux = ["$", ","]
    preco = aux[2]
    for c in str_aux:
        preco = preco.replace(c, '')
    estrutura['Cost'].append(float(preco))

In [ ]:
estrutura

In [ ]:
df = pd.DataFrame(estrutura)

In [ ]:
df

In [ ]:
df.describe()